In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
train = pd.read_csv('../input/milamlcompetition1/train.csv')
test = pd.read_csv('../input/milamlcompetition1/test.csv')
X_train = train['Abstract']
y_train = train['Category']
X_test = test['Abstract']

In [3]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

In [4]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_one_hot = onehot_encoder.fit_transform(integer_encoded)

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier

#tfidf = TfidfVectorizer(strip_accents=None,
#                        lowercase=False,
#                        preprocessor=None)

tfidf = TfidfVectorizer()


svm_tfidf = Pipeline([('vect', tfidf),
                     ('clf', SGDClassifier(loss='hinge', random_state=0))])

clf = StackingClassifier(estimators=svm_tfidf, final_estimator=LogisticRegression(random_state=0))

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

#tfidf = TfidfVectorizer(strip_accents=None,
#                        lowercase=False,
#                        preprocessor=None)

tfidf = TfidfVectorizer(stop_words = "english", sublinear_tf = True)


lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB

In [7]:
tfidf = TfidfVectorizer(stop_words = "english", sublinear_tf = True)

In [8]:
tfidf.fit(X_train)

TfidfVectorizer(stop_words='english', sublinear_tf=True)

In [9]:
aX_train = tfidf.transform(X_train)
aX_test = tfidf.transform(X_test)

In [10]:
X_tr, X_eval, y_tr ,y_eval = train_test_split(aX_train, y_train,test_size=0.15,shuffle=True,random_state=0,stratify=y_train)

In [11]:
log_r = LogisticRegression(random_state=0)
svm = SGDClassifier(loss='hinge', random_state=0)
mlp = MLPClassifier(hidden_layer_sizes = (150,), max_iter=1, batch_size=64)
multi_nb = MultinomialNB()
cnb = ComplementNB()

In [ ]:
cnb = ComplementNB()

In [ ]:
from sklearn.ensemble import VotingClassifier

ens = VotingClassifier(estimators=[ ('1',log_r), ('2',cnb), ('3',multi_nb),('4',mlp),('5',svm) ], voting='hard') #, ('2',svm) ('2',cnb)
    
ens.fit(X_tr, y_tr) 
train_score = ens.score(X_tr, y_tr)
val_score = ens.score(X_eval, y_eval)

In [12]:
from sklearn.ensemble import VotingClassifier

ens_no_nb = VotingClassifier(estimators=[ ('1',log_r),('2',mlp),('3',svm) ], voting='hard') #, ('2',cnb) ('3',multi_nb)
    
ens_no_nb.fit(X_tr, y_tr) 
train_score = ens_no_nb.score(X_tr, y_tr)
val_score = ens_no_nb.score(X_eval, y_eval)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [13]:
train_score

0.9540392156862745

In [14]:
val_score

0.8044444444444444

In [15]:
model= ens_no_nb.fit(aX_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [16]:
output=model.predict(aX_test)

In [ ]:
def create_submission(prediction):
    index = np.arange(0,len(prediction))
    result = np.column_stack((index, prediction))
    header = np.array(['Id','Category'])
    result = np.vstack((header,result))
    result = result.astype(str)
    print(result)
    np.savetxt("ensemble_nonb.csv", result, delimiter=",",  fmt='%s')

In [ ]:
create_submission(output)